In [2]:
import re

import requests
from bs4 import BeautifulSoup

#part 1: https://pumpout.anyhowstep.com/ 으로부터의 데이터 추출: metadata확보
#part 2: piugame.com의 선곡 랭킹으로부터의 데이터 추출: 작곡가/노래 이름 한-영 번역 기능 추가

%load_ext autoreload
%autoreload 2

In [248]:


#Request Soup 형성
def extractSoupfromURL(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')
    return request, soup

#해당 URL이 유효한지 판별
def check_valid_url(soup):
    if soup.find('div', {'class': "panel-heading bg bg-info"}):
        return True
    return False

#URL이 유효했는지 로그 남김
def printLog(log, url, isURLValid):
    if isURLValid:
        print(f'{url} is valid')
        log.append(f'{url} is valid')
    else:
        print(f'{url} is invalid')
        log.append(f'{url} is invalid')

#title/step_artist/levels/bpm/category/gametype 추출
def get_title(soup):
    title = soup.find('title').prettify()
    ans = title.replace("\t", "").replace("\n", "")
    return ans[17:-19]

def get_step_artist(soup):
    step_artist = r'bg-primary">(.*?)</span>'
    step_artist_list = re.findall(step_artist, str(soup))
    return step_artist_list

def get_levels(soup):
    levels = r'Levels/(.*?).png"'
    soup_partial = soup.find('div', class_='list-group-item')
    levels_list = re.findall(levels, str(soup_partial))
    ans = level_format_list(levels_list)
    return ans
def extract_before_slash(s):
    # Split the string by '/' and take the first element
    return s.split('/')[0]

def level_format(str):
    type = extract_before_slash(str)
    level = str[-2:]
    return f'{type}{level}'

def level_format_list(levels):
    ans = []
    for i in levels:
        new_item = level_format(i)
        ans.append(new_item)
    print(ans)
    return ans

def get_bpm(soup):
    span_text = soup.find('span', {'class': 'badge bg-info'}).text
    bpm = ''.join(filter(str.isdigit, span_text))
    return bpm

def get_category(soup):
    category = soup.find('span', {'class': 'badge bg-success pull-right'}).text
    return category

def get_gametype(soup):
    gametype = soup.find('span', {'class': 'badge bg-inverse pull-right'}).text
    return gametype

#metadata 추출 - 위 function들을 하나로 합침
def extract_metadata(soup):
    title = get_title(soup)
    step_artist_list = get_step_artist(soup)
    levels_list = get_levels(soup)
    bpm = get_bpm(soup)
    category = get_category(soup)
    gametype = get_gametype(soup)
    return title, step_artist_list, levels_list, bpm, category, gametype
    

In [249]:
#metadata를 string으로 변환
def string_of_data(index, valid, Song, BPM, Category, GameType, Level, StepArtist):
    if valid == False:
        return f'index = {index} is invalid\n-\n'
    else:
        return f'index = {index}\nSong: {Song}\nBPM: {BPM}\nCategory: {Category}\nGameType: {GameType}\nLevel: {Level}\nStepArtist: {StepArtist}\n-\n'

#Utility Function: 혹시 스텝-제작자가 매칭이 되지 않을지 확인
def checkLength(list1, list2):
    if len(list1) != len(list2):
        print(f'Length of Levels: {len(list1)}\nLength of StepArtists: {len(list2)}')
    
#URL을 통해 soup을 추출하고, 유효한지 판별
def parse_URL(url_num):
    current_URL = f'https://pumpout.anyhowstep.com/songs/{url_num}'
    request, soup = extractSoupfromURL(current_URL)
    isURLValid = check_valid_url(soup)
    return current_URL, request, soup, isURLValid

#list를 string으로 변환
def get_log_string(log):
    ans = ""
    for i in log:
        ans += i + "\n"
    return ans


In [ ]:
#initialize
url_list = []
requests_list = []
soup_list = []
url_valid_list = []
log = []

for i in range(929):
    current_URL, request, soup, isURLValid = parse_URL(i+1)
    #Get URL, Request,Soup, and check if URL is valid
    url_list.append(current_URL)
    requests_list.append(request)
    soup_list.append(soup)
    isURLValid = check_valid_url(soup_list[i])
    url_valid_list.append(isURLValid)
    #printLog(log, url_list[i], isURLValid)



In [254]:
#먼저 앞으로 URL에 다시 접근하지 않아도 되도록 먼저 soup를 txt 파일로 백업했음
def BigURLString(soup_list):
    ans = ""
    for i in range(len(soup_list)):
        unitStr = f"!!URL index= {i}!!\n"
        unitStr += soup_list[i].prettify()
        unitStr += "!!End of URL!!\n\n"
        ans += unitStr
    print(i)
    return ans

strSoup = BigURLString(soup_list)

with open('Soup_backup.txt', 'w', encoding = 'utf-8') as f:
    f.write(strSoup)
    f.close()

928


In [ ]:
#metadata의 종류들을 initialize
Song_list = []
BPM_list = []
Category_list = []
Level_list = []
StepArtist_list = []
Gametype_list = []
#TODO: 출시 버전 정리

#metadata 추출
for i in range(len(soup_list)):
    if url_valid_list[i] == True:
        #if URL is valid, then start parsing.
        song_title, step_artist_list, levels_list, bpm, category, gametype = extract_metadata(soup_list[i])
        Song_list.append(song_title)
        BPM_list.append(bpm)
        Category_list.append(category)
        Level_list.append(levels_list)
        StepArtist_list.append(step_artist_list)
        Gametype_list.append(gametype)
    else:
        print(f'{url_list[i]} is invalid. Skipping...')


In [259]:
#로그 정리 (URL 파싱 성공 여부)
Writeme = ""
for i in range(915):
    Writeme += string_of_data(i, url_valid_list[i], Song_list[i], BPM_list[i], Category_list[i], Gametype_list[i], Level_list[i], StepArtist_list[i])

log_result = get_log_string(log)

In [260]:
#txt 파일로 저장
with open('output.txt', 'w') as f:
    f.write(Writeme)
    f.close()

with open('output_log.txt', 'w') as f:
    f.write(log_result)
    f.close()

In [103]:
with open('SongRanks-KR.txt', encoding = 'utf-8') as file:
    text2 = file.read()

with open('SongRanks-EN.txt', encoding = 'utf-8') as file:
    text1 = file.read()
    
def remove_tabs_and_enters(text):
    return text.replace('\t', '').replace('\n', '')

def remove_multiple_spaces(text):
    return ' '.join(text.split())

#URL 파싱 과정에서의 anomalie들 정리
def correct_anomalies_html(text):
    text = text.replace('&amp;', '&')
    text = text.replace('&quot;', '"')
    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    text = text.replace('&nbsp;', ' ')
    return text

#URL 파싱 과정에서의 anomalie들 정리: 특정 노래들에 대해서 정리
def correct_anomalies_custom(text):
    text = text.replace("feat.  KuTiNA", "feat. KuTiNA")
    return text

In [104]:
def extract_content_between_tags(text):
    # 정규 표현식을 사용하여 "t1">과 </p> 사이의 내용을 찾음
    pattern1 = r't1">(.*?)</p>'
    pattern2 = r't2">(.*?)</p>'
    # findall 메서드를 사용해 모든 일치 항목을 리스트로 반환
    matches1 = re.findall(pattern1, text, re.DOTALL)
    matches2 = re.findall(pattern2, text, re.DOTALL)
    return matches1, matches2
text1 = correct_anomalies_custom(correct_anomalies_html(remove_multiple_spaces(remove_tabs_and_enters(text1))))
text2 = correct_anomalies_custom(correct_anomalies_html(remove_multiple_spaces(remove_tabs_and_enters(text2))))
songname_EN, composer_EN = extract_content_between_tags(text1)
songname_KR, composer_KR = extract_content_between_tags(text2)

with open('MetaData-ComposerSongNames.txt', 'w', encoding = 'utf-8') as file:
    for i in range(544):
        file.write(f'{songname_KR[i]}//{composer_KR[i]}//{songname_EN[i]}//{composer_EN[i]}\n')


In [ ]:

#혹시 double space가 더 남아있는기 검증했었음
def detect_double_spaces(text):
    return '  ' in text
'''
with open('MetaData-ComposerSongNames.txt', encoding = 'utf-8') as file:
    text_metadata = file.read()

print(detect_double_spaces(text_metadata))
'''

In [14]:
def parse_back_to_soups(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Split the content by the starting delimiter (ignoring the first split which is empty)
    segments = content.split("!!URL index=")[1:]
    
    # Extract the HTML content from each segment
    html_contents = []
    for segment in segments:
        # Further split by the ending delimiter and take the first part
        html_content = segment.split("!!End of URL!!\n")[0]
        # The actual content starts after the first newline character
        html_content = html_content.split('\n', 1)[1]
        html_contents.append(html_content.strip())
    
    return html_contents

# Example usage
filename = "Soup_backup.txt"
html_contents = parse_back_to_soups(filename)

# Optionally, convert each HTML string back to a BeautifulSoup object
soup_objects = [BeautifulSoup(html, 'html.parser') for html in html_contents]

# Do something with your soup objects
soup_objects[0].prettify()

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   Songs\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t&gt;\t\t\tAll I Want For X-mas\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\n - Pump Out\n  </title>\n  <link href="/images/9x9.gif" rel="icon" type="image/png"/>\n  <meta content="Pump Out is a searchable database for Pump it Up!" name="description"/>\n  <meta content="\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSongs\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t&gt;\t\t\tAll I Want For X-mas\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\n - Pump Out" itemprop="name">\n   <meta content="http://pumpout.anyhowstep.com//images/TitleCards/20.png" itemprop="image"/>\n   <meta content="Pump Out is a 

In [194]:
from piu_utilities_crawling import *

#get Chimera's metadata

testme = soup_objects[829]

txt = extract_metadata(testme)

txt

New Gangsta
## Adrenaline Blaster ##
bpm: 150
category: ORIGINAL
gametype: Arcade
Fine Here
Fine THere
Fine Everywhere


TypeError: 'str' object is not callable

In [141]:
sample = target_div[5]

def level_format(str):
    parts = str.split("Levels/")
    substring_after_levels = parts[1] if len(parts) > 1 else None
    substring_after_levels = extract_before_slash(substring_after_levels)
    if str[-8:-6] == "sp":
        level = "??"
    else:
        level = str[-6:-4]
    return f'{substring_after_levels}{level}'

def parse_version_and_detail(str):
    parts = str.split(" - ")
    if len(parts) != 2:
        return "ERROR", "ERROR"
    else:
        return parts[0].strip(), parts[1].strip()

def parse_stepLV_artist_version(sample):
    
    img_elements = sample.find_all('img')

    level = level_format(img_elements[0]['src'])
    #print(level)
    level_history = []
    step_artist_list = []
    
    for item in img_elements[1:]:
        level_history.append(level_format(item['src']))
    #print(level_history)

    step_artist_list = sample.find_all('span', {'class': 'label bg-primary'})
    for index in range(len(step_artist_list)):
        step_artist_list[index] = step_artist_list[index].text.strip()
    version, version_detail = parse_version_and_detail(sample.find('span', {'class': 'label bg-info'}).text.strip())
    #print(step_artist_list, version, version_detail)

    tags = []


    return level, level_history, step_artist_list, version, version_detail, tags
    #return level, level_history, step_artist, version, version_detail
parse_stepLV_artist_version(sample)

for item in target_div:
    print(parse_stepLV_artist_version(item))


('S04', [], ['ANDAMIRO'], 'Fiesta', 'v1.01')
('S09', ['S07', 'S08'], ['JKD'], 'NX / New Xenesis', 'default')
('S13', ['S12', 'S11'], ['ANDAMIRO'], 'Fiesta', 'v1.01')
('S14', ['S16', 'S13'], ['BPM'], 'NX / New Xenesis', 'default')
('S18', [], ['BPM'], 'Fiesta', 'v1.01')
('S19', ['S16', 'S17'], ['ANDAMIRO', 'AHGOO'], 'NX / New Xenesis', 'default')
('S19', ['S20'], ['BPM'], 'NX2 / Next Xenesis', 'default')
('S20', [], ['ESC'], 'ERROR', 'ERROR')
('S23', ['S22'], ['EHNE'], 'NX / New Xenesis', 'default')
('D18', ['D16', 'D15'], ['BPM'], 'NX / New Xenesis', 'default')
('D21', ['D20'], ['WING'], 'NX / New Xenesis', 'default')
('D22', [], ['OSING'], 'XX', 'v2.03.0')
('D26', ['D25', 'D24'], ['EHNE'], 'NX / New Xenesis', 'default')
('D??', ['R16'], ['BPM'], 'NX / New Xenesis', 'default')
('C02', ['R10', 'DP??'], ['ANDAMIRO'], 'Fiesta', 'v1.01')


In [117]:
span_elements = sample.find_all('span')
span_elements

[<span class="label bg-primary">
             ANDAMIRO
            </span>,
 <span class="label bg-primary">
             AHGOO
            </span>,
 <span class="label bg-info">
 <i class="fa fa-plus">
 </i>
             NX / New Xenesis - default
            </span>,
 <span class="label bg-inverse">
             HARD
            </span>,
 <span class="label bg-inverse">
             UPDATED
            </span>]

In [ ]:
def get_step_artist_test(soup):
    target_div = soup.find('div', class_='col-lg-6').find('section', class_='panel').find('div', class_='list-group').find_all('div', class_="media")
    img_elements = target_div.find_all('img')
    span_text = soup.find_all('span', {'class': 'badge bg-info'}).text
    filtered_imgs = [img for img in img_elements if img.get('class') == ['thumb', 'pull-left'] and img['src'].startswith("/images/Levels")]

    bpm = ''.join(filter(str.isdigit, span_text))
    #print('bpm:', bpm)
    return filtered_imgs


target_div = testme.find('div', class_='col-lg-6').find('section', class_='panel').find('div', class_='list-group').find_all('div', class_="media")
#target_div = target_div.find_all('div', class_="media")


img_elements = target_div.find_all('img')
img_elements
span_elements = target_div.find_all('span')
# Filter imgs based on class and the starting part of 'src'
filtered_imgs = [img for img in img_elements if img.get('class') == ['thumb', 'pull-left'] and img['src'].startswith("/images/Levels")]

sample = target_div[5]

level = 0
step_type = ""
version = ""
level_history = []
tags = []

img_elements = sample.find_all('img')

span_elements = sample.find_all('span')

img_elements

In [89]:
span_elements



[<span class="label bg-primary">
             ANDAMIRO
            </span>,
 <span class="label bg-primary">
             AHGOO
            </span>,
 <span class="label bg-info">
 <i class="fa fa-plus">
 </i>
             NX / New Xenesis - default
            </span>,
 <span class="label bg-inverse">
             HARD
            </span>,
 <span class="label bg-inverse">
             UPDATED
            </span>]

In [80]:
filtered_spans = [span.get_text(strip=True) for span in span_elements if span.get('class') == ['label', 'bg-primary']]


print(len(filtered_imgs))
print(len(filtered_spans))


filtered_spans

15

In [76]:
import json

class Tire:
    def __init__(self, diameter, material):
        self.diameter = diameter
        self.material = material
    
    def to_dict(self):
        return {
            "diameter": self.diameter,
            "material": self.material
        }

class Car:
    manufacturer = "Toyota"
    
    def __init__(self, model, color, year, tires, owners):
        self.model = model
        self.color = color
        self.year = year
        self.tires = tires  # Tire 인스턴스의 리스트
        self.owners = owners  # 소유자 이름의 리스트
    
    def to_json(self):
        data = {
            "manufacturer": Car.manufacturer,
            "model": self.model,
            "color": self.color,
            "year": self.year,
            "tires": [tire.to_dict() for tire in self.tires],
            "owners": self.owners  # 소유자 정보 추가
        }
        return json.dumps(data, indent=4)

# Tire 인스턴스 생성
tires = [Tire(16, "Rubber"), Tire(16, "Rubber")]

# 소유자 리스트
owners = ["John Doe", "Jane Doe"]

# Car 인스턴스 생성 시 타이어와 소유자 정보 포함
car = Car("Camry", "Black", 2019, tires, owners)

# Car 인스턴스를 JSON으로 변환하여 출력
print(car.to_json())


{
    "manufacturer": "Toyota",
    "model": "Camry",
    "color": "Black",
    "year": 2019,
    "tires": [
        {
            "diameter": 16,
            "material": "Rubber"
        },
        {
            "diameter": 16,
            "material": "Rubber"
        }
    ],
    "owners": [
        "John Doe",
        "Jane Doe"
    ]
}


In [180]:
#VersionData:
def get_normal_float(value):
    flag = 0
    for i in range(len(value)):
        if value[i] == '.':
            flag += 1
    if flag > 1:
        return value.replace('.', '', 1)
    else:
        return value

version = "2.03.1"



print(get_normal_float(version))  # Output: 2.031

203.1


In [195]:
str = "_10"
str2 = "21"

str+ str2

'_1021'